In [5]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
train_df = pd.read_csv('train_mix.csv')
columns_to_drop = ["Unnamed: 0", "ID"]
df_cleaned = train_df.drop(columns=columns_to_drop).dropna(axis=1)
train, test = train_test_split(df_cleaned, test_size=0.2, random_state=42)
train, validation = train_test_split(train, test_size=0.2, random_state=42) 

In [7]:
X_train = train.drop(columns=['Status']).values
y_train = train['Status'].values
X_val = validation.drop(columns=['Status']).values
y_val = validation['Status'].values
X_test = test.drop(columns=['Status']).values
y_test = test['Status'].values

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)

#  a simple MLP model
class MLP(nn.Module):
    def __init__(self, layer_sizes):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes) - 2:
                layers.append(nn.ReLU())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Step 7: Grid Search Setup
hidden_units_options = [[64], [128], [64, 64], [128, 64]]
learning_rates = [0.001, 0.01, 0.1]
results = []

num_epochs = 500
patience = 50  # For early stopping

# Grid Search Loop
for hidden_units in hidden_units_options:
    for lr in learning_rates:
        model = MLP([X_train_tensor.shape[1]] + hidden_units + [4]).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        best_val_loss = float('inf')
        patience_counter = 0

        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train_tensor.to(device))
            loss = criterion(outputs, y_train_tensor.to(device))
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                val_outputs = model(X_val_tensor.to(device))
                val_loss = criterion(val_outputs, y_val_tensor.to(device))

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                break

        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor.to(device))
            _, test_predictions = torch.max(test_outputs, 1)
            test_accuracy = accuracy_score(y_test, test_predictions.cpu())

        # Save results
        results.append({
            'hidden_units': str(hidden_units),
            'learning_rate': lr,
            'test_accuracy': test_accuracy,
        })

results_df = pd.DataFrame(results)
print(results_df)
# results_df.to_csv('grid_search.csv', index=False)

# print("Grid search completed. Results saved to grid_search.csv.")

   hidden_units  learning_rate  test_accuracy
0          [64]          0.001       0.785714
1          [64]          0.010       0.750000
2          [64]          0.100       0.785714
3         [128]          0.001       0.809524
4         [128]          0.010       0.750000
5         [128]          0.100       0.702381
6      [64, 64]          0.001       0.797619
7      [64, 64]          0.010       0.738095
8      [64, 64]          0.100       0.738095
9     [128, 64]          0.001       0.785714
10    [128, 64]          0.010       0.773810
11    [128, 64]          0.100       0.797619
